# Libraries

In [251]:
import pandas as pd
from getpass import getuser


In [252]:
user = getuser()

# input datasets

In [253]:
github_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/single matches/"
website_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/website_df.xlsx"
website_df = pd.read_excel(website_path)

In [254]:
# Create the single_matches_web dataframe by selecting rows where p2t1 and p2t2 are empty
single_matches_web_df = website_df[website_df['p2t1'].isna() & website_df['p2t2'].isna()]
double_matches_web_df = website_df[website_df['p2t1'].notna() & website_df['p2t2'].notna()]
print(website_df.shape)
print(single_matches_web_df.shape)
print(double_matches_web_df.shape)

(494, 19)
(387, 19)
(107, 19)


# single_matches_web_df

Daniel Brands (Germany) vs Roberto Bautista Agut (Spain) is recorded as match number 4 in website_df since in the match before Feliciano Lopez (Spain) walked over Philipp Kohlschreiber (Germany). In website_df, we therefore consider it as a match not played whereas in github_df the number of match is preserved. We therefore change match number in website_df to make it consistent. 

In [255]:
# Change the number of match  
single_matches_web_df.loc[
    (single_matches_web_df['year'] == 2014) &
    (single_matches_web_df['team1'] == 'germany') &
    (single_matches_web_df['team2'] == 'spain') &
    (single_matches_web_df['match'] == 4), 'match'] = 5



# github data

In [256]:
# Range of years to include
years = range(2013, 2024)
dfs = []

for year in years:
    atp_matches_path = f"{github_path}atp_matches_{year}.csv"
    df = pd.read_csv(atp_matches_path)
    dfs.append(df)

# Concatenating all the DataFrames into one
atp_matches_df = pd.concat(dfs, ignore_index=True)

# Extract the year from the 'tourney_id' by splitting the string at the first hyphen and taking the first part
atp_matches_df['year'] = atp_matches_df['tourney_id'].str.split('-').str[0]

# Convert the 'year' column to an integer
atp_matches_df['year'] = atp_matches_df['year'].astype(int)

# Replacing "Frances Tiafoe" with "Franklin Tiafoe" in both the 'winner_name' and 'loser_name' columns
atp_matches_df['winner_name'] = atp_matches_df['winner_name'].replace('Frances Tiafoe', 'Franklin Tiafoe')
atp_matches_df['loser_name'] = atp_matches_df['loser_name'].replace('Frances Tiafoe', 'Franklin Tiafoe')

# Replacing "Andreas Haider Maurer" with "Andreas Haider-Maurer" in both the 'winner_name' and 'loser_name' columns
atp_matches_df['winner_name'] = atp_matches_df['winner_name'].replace('Andreas Haider Maurer', 'Andreas Haider-Maurer')
atp_matches_df['loser_name'] = atp_matches_df['loser_name'].replace('Andreas Haider Maurer', 'Andreas Haider-Maurer')

# Replacing "Guillermo Garcia Lopez" with "Guillermo Garcia-Lopez" in both the 'winner_name' and 'loser_name' columns
atp_matches_df['winner_name'] = atp_matches_df['winner_name'].replace('Guillermo Garcia Lopez', 'Guillermo Garcia-Lopez')
atp_matches_df['loser_name'] = atp_matches_df['loser_name'].replace('Guillermo Garcia Lopez', 'Guillermo Garcia-Lopez')

# Save the combined data to a single Excel file
final_output_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/single matches/atp_matches_df.xlsx"
atp_matches_df.to_excel(final_output_path, index=False)


# github repo matched observations with website df players

In [257]:
# Initialize a list to store all ATP matches where Davis Cup players took part
combined_matches_list = []

# Iterate through each year to filter the matches
for year in range(2013, 2024):
    # Filter Davis Cup data for the current year
    davis_single_year_df = single_matches_web_df[single_matches_web_df['year'] == year]
    
    # Create a set of tuples representing pairs of players for the current year
    davis_pairs = set(zip(davis_single_year_df['p1t1'], davis_single_year_df['p1t2']))
    
    # Filter ATP matches data to match Davis Cup pairs for the current year
    matched_atp_matches_df = atp_matches_df[
        (atp_matches_df['year'] == year) &
        atp_matches_df.apply(lambda row: (row['winner_name'], row['loser_name']) in davis_pairs or
                                           (row['loser_name'], row['winner_name']) in davis_pairs, axis=1)
    ]
    
    # Append the filtered data to the list
    combined_matches_list.append(matched_atp_matches_df)

# Concatenate all the DataFrames in the list into one combined DataFrame
combined_matches_df = pd.concat(combined_matches_list, ignore_index=True)

# Save the combined data to a single Excel file
final_output_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/single matches/davis_cup_players.xlsx"
combined_matches_df.to_excel(final_output_path, index=False)


In [258]:
# Count the number of observations where tourney_level is "D" - Davis Cup
davis_single_github_count = combined_matches_df[combined_matches_df['tourney_level'] == 'D'].shape[0]

# Count the number of observations where tourney_level is not "D"
extra_matches_single_count = combined_matches_df[combined_matches_df['tourney_level'] != 'D'].shape[0]

print("Number of observations for davis cup matches from github", davis_single_github_count)
print("Number of observations for matches played by davis cup players outside davis cup:", extra_matches_single_count)


Number of observations for davis cup matches from github 387
Number of observations for matches played by davis cup players outside davis cup: 131


130 is the number of games played in other atp matches by davis cup players

# github dataset single match players

In [259]:
# Creating a new DataFrame with observations where tourney_level is "D"
single_github_df = combined_matches_df[combined_matches_df['tourney_level'] == 'D']
single_github_df.shape

(387, 50)

we have 387 observations for single davis matches for both single_github_df and single_matches_web_df

# list from github dataset

In [260]:

# Creating a list with year, match_num, winner_name, loser_name, where names are alphabetically ordered
github_list = single_github_df.apply(
    lambda row: [row['year'],row['match_num'], *sorted([row['winner_name'], row['loser_name']])], 
    axis=1
).tolist()

# Optionally, print the first few elements in the list to verify
print(github_list[:5])


[[2013, 1, 'Carlos Berlocq', 'Philipp Kohlschreiber'], [2013, 2, 'Florian Mayer', 'Juan Monaco'], [2013, 4, 'Juan Monaco', 'Tobias Kamke'], [2013, 5, 'Carlos Berlocq', 'Christopher Kas'], [2013, 1, 'David Goffin', 'Viktor Troicki']]


# list from website df

In [261]:


# Creating a list with Year, cleaned Match, P1T1, P1T2, where player names are alphabetically ordered
website_list = single_matches_web_df.apply(
    lambda row: [row['year'], row['match'], *sorted([row['p1t1'], row['p1t2']])],
    axis=1
).tolist()

# Optionally, print the first few elements in the list to verify
print(website_list[:5])




[[2013, 1, 'Albert Ramos', 'Milos Raonic'], [2013, 2, 'Frank Dancevic', 'Marcel Granollers'], [2013, 4, 'Guillermo Garcia-Lopez', 'Milos Raonic'], [2013, 5, 'Albert Ramos', 'Frank Dancevic'], [2013, 1, 'Marin Cilic', 'Paolo Lorenzi']]


In [262]:
# Standardizing formats in matches_list and davis_single_list
# Assuming year and match number are integers, and player names are strings

standardized_github_list = [
    [int(year), int(match_num), str(winner_name), str(loser_name)]
    for year, match_num, winner_name, loser_name in github_list
]

standardized_web_list = [
    [int(year),int(match_num), str(player1), str(player2)]
    for year, match_num, player1, player2 in website_list
]

# Convert to sets of tuples for comparison
standardized_github_set = set(tuple(item) for item in standardized_github_list)
standardized_web_set = set(tuple(item) for item in standardized_web_list)

# Find non-matching elements in both sets
yes_github_no_web = standardized_github_set - standardized_web_set
no_github_yes_web = standardized_web_set - standardized_github_set

print("Non-matching elements in standardized_web_list:", yes_github_no_web)
print("Non-matching elements in standardized_github_list:", no_github_yes_web)

# Count the number of non-matching tuples in standardized_github_list
num_yes_github_no_web = len(yes_github_no_web)

# Count the number of non-matching tuples in standardized_web_list
num_no_github_yes_web = len(no_github_yes_web)

print("Number of unmatched elements in num_yes_github_no_web:", num_yes_github_no_web)
print("Number of unmatched elements in num_no_github_yes_web:", num_no_github_yes_web)


Non-matching elements in standardized_web_list: set()
Non-matching elements in standardized_github_list: set()
Number of unmatched elements in num_yes_github_no_web: 0
Number of unmatched elements in num_no_github_yes_web: 0


# merge two datasets single_github_df and single_matches_web_df

In [266]:
# Ensure 'year' is an integer in single_github_df
single_github_df['year'] = single_github_df['year'].astype(int)

# Ensure 'year' is an integer in single_matches_web_df
single_matches_web_df['year'] = single_matches_web_df['year'].astype(int)

# Standardize case and remove leading/trailing spaces in single_github_df
single_github_df['winner_name'] = single_github_df['winner_name'].str.lower().str.strip()
single_github_df['loser_name'] = single_github_df['loser_name'].str.lower().str.strip()

# Standardize case and remove leading/trailing spaces in single_matches_web_df
single_matches_web_df['p1t1'] = single_matches_web_df['p1t1'].str.lower().str.strip()
single_matches_web_df['p1t2'] = single_matches_web_df['p1t2'].str.lower().str.strip()

# Now perform the merges as described

# Direct match merge
merged_df = pd.merge(
    single_github_df, 
    single_matches_web_df, 
    how='left', 
    left_on=['year', 'winner_name', 'loser_name'], 
    right_on=['year', 'p1t1', 'p1t2'],
    indicator=True  # This will add a column to indicate if the merge was successful
)

# Reverse match merge (loser_name vs. winner_name)
merged_df_reverse = pd.merge(
    single_github_df, 
    single_matches_web_df, 
    how='left', 
    left_on=['year', 'loser_name', 'winner_name'], 
    right_on=['year', 'p1t1', 'p1t2'],
    indicator=True  # This will add a column to indicate if the merge was successful
)

# Combine both merged DataFrames
combined_merged_df = pd.concat([merged_df, merged_df_reverse], ignore_index=True)

# Drop duplicates if any
combined_merged_df = combined_merged_df.drop_duplicates()

# Check for unmatched rows in combined_merged_df
unmatched_rows_df = combined_merged_df[combined_merged_df['_merge'] == 'left_only']

# Count the number of unmatched rows
num_unmatched = unmatched_rows_df.shape[0]

print(f"Number of unmatched rows in combined_merged_df: {num_unmatched}")

# If needed, you can inspect these unmatched rows
if num_unmatched > 0:
    print("Unmatched rows:")
    print(unmatched_rows_df)


Number of unmatched rows in combined_merged_df: 387
Unmatched rows:
                          tourney_id                     tourney_name surface  \
4                          2013-D003      Davis Cup WG R1: BEL vs SRB    Clay   
5                          2013-D003      Davis Cup WG R1: BEL vs SRB    Clay   
11                         2013-D001      Davis Cup WG R1: CAN vs ESP    Hard   
16                         2013-D002      Davis Cup WG R1: ITA vs CRO    Clay   
18                         2013-D002      Davis Cup WG R1: ITA vs CRO    Clay   
..                               ...                              ...     ...   
764  2023-M-DC-2023-FLS-M-CZE-AUS-01  Davis Cup Finals QF: CZE vs AUS    Hard   
769  2023-M-DC-2023-FLS-M-ITA-NED-01  Davis Cup Finals QF: ITA vs NED    Hard   
771  2023-M-DC-2023-FLS-M-ITA-SRB-01  Davis Cup Finals SF: ITA vs SRB    Hard   
772  2023-M-DC-2023-FLS-M-SRB-GBR-01  Davis Cup Finals QF: SRB vs GBR    Hard   
773  2023-M-DC-2023-FLS-M-SRB-GBR-01  Dav

C:\Users\aldi\AppData\Local\Temp\ipykernel_17836\3206966950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_github_df['year'] = single_github_df['year'].astype(int)
C:\Users\aldi\AppData\Local\Temp\ipykernel_17836\3206966950.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_matches_web_df['year'] = single_matches_web_df['year'].astype(int)
C:\Users\aldi\AppData\Local\Temp\ipykernel_17836\3206966950.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

# Exporting dataframes

In [265]:
file_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/unmatched_rows_df.xlsx"

# Export the DataFrame to an Excel file
unmatched_rows_df.to_excel(file_path, index=False)

print(f"DataFrame exported to {file_path}")

file_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/single_matches_web.xlsx"

# Export the DataFrame to an Excel file
single_matches_web.to_excel(file_path, index=False)

print(f"DataFrame exported to {file_path}")


DataFrame exported to C:/Users/aldi/Documents/GitHub/davis-cup/data/unmatched_rows_df.xlsx
DataFrame exported to C:/Users/aldi/Documents/GitHub/davis-cup/data/single_matches_web.xlsx


filtered_atp_matches.xlsx in single matches folder = final df for single matches

# Double matches

In [125]:

# Sample setup - ensure to replace this with actual data loading logic
# df_paired = pd.read_csv('path_to_davis_cup_data.csv')

folder_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/double matches/"

double_matches_df = pd.DataFrame()

for year in range(2014, 2021):
    davis_double_year_df = davis_double_df[davis_double_df['year'] == year].copy()

    # Generate sorted tuples for each Davis Cup match
    davis_quads = set(
        davis_double_year_df.apply(
            lambda x: (
                tuple(sorted([str(x['p1t1']), str(x['p2t1'])])),
                tuple(sorted([str(x['p1t2']), str(x['p2t2'])]))
            ), axis=1
        )
    )

    atp_matches_path = f"{folder_path}atp_matches_doubles_{year}.csv"
    atp_matches_df = pd.read_csv(atp_matches_path)

    # Filter out any rows that might have NaN values for player names to avoid type errors
    atp_matches_df.dropna(subset=['winner1_name', 'winner2_name', 'loser1_name', 'loser2_name'], inplace=True)

    # Create sorted tuples for each ATP match
    matched_atp_matches_df = atp_matches_df[
        atp_matches_df.apply(
            lambda row: (
                tuple(sorted([str(row['winner1_name']), str(row['winner2_name'])])),
                tuple(sorted([str(row['loser1_name']), str(row['loser2_name'])]))
            ) in davis_quads, axis=1
        )
    ]

    # Append matched ATP matches to the final DataFrame
    double_matches_df = pd.concat([double_matches_df, matched_atp_matches_df], ignore_index=True)

if not double_matches_df.empty:
    double_matches_df['year'] = double_matches_df['tourney_id'].astype(str).str.split('-').str[0].astype(int)
    final_output_path = "C:/Users/{}/Documents/GitHub/davis-cup/data/double matches/filtered_atp_matches_doubles.xlsx"
    double_matches_df.to_excel(final_output_path, index=False)
else:
    print("No matches found or 'tourney_id' column missing in all years.")


No matches found or 'tourney_id' column missing in all years.
